In [2]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 1 : Imports et Configuration
# ════════════════════════════════════════════════════════════════════

import boto3
from botocore.exceptions import ClientError
import os
from pathlib import Path
from dotenv import load_dotenv
import json

load_dotenv('../config/.env')

# Configuration
BUCKET_NAME = os.getenv('AWS_S3_BUCKET')
REGION = os.getenv('AWS_REGION')

# Client S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=REGION
)

print("✅ Configuration S3 chargée")
print(f"   • Bucket : {BUCKET_NAME}")
print(f"   • Région : {REGION}")

✅ Configuration S3 chargée
   • Bucket : 251107-140505-jedha-kayak-project
   • Région : eu-west-3


In [ ]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 2 : Vérification du Bucket Existant
# ════════════════════════════════════════════════════════════════════

print("📦 VÉRIFICATION DU BUCKET S3\n")
print("="*60)

try:
    # Vérifier que le bucket existe
    s3_client.head_bucket(Bucket=BUCKET_NAME)
    print(f"✅ Bucket '{BUCKET_NAME}' accessible")
    
    # Récupérer les informations du bucket
    try:
        location = s3_client.get_bucket_location(Bucket=BUCKET_NAME)
        region = location['LocationConstraint'] or 'us-east-1'
        print(f"   • Région : {region}")
    except:
        pass
    
    # Lister les fichiers existants
    try:
        response = s3_client.list_objects_v2(Bucket=BUCKET_NAME, MaxKeys=100)
        
        if 'Contents' in response:
            total_size = sum(obj['Size'] for obj in response['Contents'])
            print(f"   • Fichiers existants : {response['KeyCount']}")
            print(f"   • Taille totale : {total_size / (1024*1024):.2f} MB")
            
            # Afficher quelques fichiers
            if response['KeyCount'] > 0:
                print(f"\n📁 Aperçu des fichiers existants :")
                for obj in response['Contents'][:5]:
                    size_kb = obj['Size'] / 1024
                    print(f"   • {obj['Key']:50s} ({size_kb:.1f} KB)")
                
                if response['KeyCount'] > 5:
                    print(f"   ... et {response['KeyCount'] - 5} autres fichiers")
        else:
            print(f"   • Fichiers existants : 0")
            print(f"   📭 Le bucket est vide")
    
    except ClientError as e:
        print(f"   ⚠️  Impossible de lister les fichiers : {e}")

except ClientError as e:
    error_code = e.response['Error']['Code']
    
    if error_code == '404':
        print(f"❌ Le bucket '{BUCKET_NAME}' n'existe pas !")
        print("\n💡 Créez-le manuellement dans la console AWS ou vérifiez le nom dans .env")
    elif error_code == '403':
        print(f"❌ Accès refusé au bucket '{BUCKET_NAME}'")
        print("\n💡 Vérifiez les permissions IAM de votre utilisateur AWS")
    else:
        print(f"❌ Erreur : {e}")


📦 VÉRIFICATION DU BUCKET S3

✅ Bucket '251107-140505-jedha-kayak-project' accessible
   • Région : eu-west-3
   • Fichiers existants : 0
   📭 Le bucket est vide


In [5]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 3 : Configuration des Permissions Publiques
# ════════════════════════════════════════════════════════════════════

print("🔓 CONFIGURATION DES PERMISSIONS PUBLIQUES")
print("="*60)

# Politique de bucket pour accès public en lecture
bucket_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "PublicReadGetObject",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": f"arn:aws:s3:::{BUCKET_NAME}/*"
        }
    ]
}

print("\n⚙️  Application de la politique d'accès public...")

try:
    # Désactiver le blocage d'accès public
    s3_client.put_public_access_block(
        Bucket=BUCKET_NAME,
        PublicAccessBlockConfiguration={
            'BlockPublicAcls': False,
            'IgnorePublicAcls': False,
            'BlockPublicPolicy': False,
            'RestrictPublicBuckets': False
        }
    )
    print("   ✅ Blocage d'accès public désactivé")
    
    # Appliquer la politique
    s3_client.put_bucket_policy(
        Bucket=BUCKET_NAME,
        Policy=json.dumps(bucket_policy)
    )
    print("   ✅ Politique d'accès public appliquée")
    print("\n💡 Les fichiers uploadés seront accessibles publiquement via URL")
    
except ClientError as e:
    error_code = e.response['Error']['Code']
    
    if error_code == 'AccessDenied':
        print(f"   ⚠️  Permissions insuffisantes pour modifier la politique")
        print("   💡 Votre utilisateur IAM doit avoir : s3:PutBucketPolicy")
        print("   📝 Les fichiers seront uploadés mais ne seront peut-être pas publics")
    else:
        print(f"   ⚠️  Erreur : {e}")

🔓 CONFIGURATION DES PERMISSIONS PUBLIQUES

⚙️  Application de la politique d'accès public...
   ✅ Blocage d'accès public désactivé
   ✅ Politique d'accès public appliquée

💡 Les fichiers uploadés seront accessibles publiquement via URL


In [6]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 4 : Liste des Fichiers à Uploader
# ════════════════════════════════════════════════════════════════════

print("📋 PRÉPARATION DES FICHIERS À UPLOADER \n")
print("="*60)

# Liste des fichiers à uploader
files_to_upload = [
    # Données CSV
    ('../data/processed/final_recommendations.csv', '../data/final_recommendations.csv'),
    ('../data/processed/top20_recommendations.csv', '../data/top20_recommendations.csv'),
    ('../data/processed/city_weather_scores.csv', '../data/city_weather_scores.csv'),
    ('../data/processed/top5_destinations.csv', '../data/top5_destinations.csv'),
    
    # CSV brut (optionnel)
    ('../data/raw/hotels_top5_all.csv', '../data/raw/hotels_top5_all.csv'),
    
    # Visualisations
    ('../data/processed/rapport_final.html', '../rapport_final.html'),
    ('../data/processed/carte_tous_hotels.html', '../cartes/carte_tous_hotels.html'),
    ('../data/processed/carte_top20.html', '../cartes/carte_top20.html'),
    ('../data/processed/dashboard_complet.png', '../images/dashboard_complet.png'),
    ('../data/processed/analysis_fusion.png', '../images/analysis_fusion.png'),
]

# Vérifier quels fichiers existent
existing_files = []
missing_files = []

print("\n🔍 Vérification des fichiers locaux :\n")

for local_path, s3_key in files_to_upload:
    if Path(local_path).exists():
        size_kb = Path(local_path).stat().st_size / 1024
        print(f"   ✅ {local_path:55s} ({size_kb:.1f} KB)")
        existing_files.append((local_path, s3_key))
    else:
        print(f"   ❌ {local_path:55s} (MANQUANT)")
        missing_files.append(local_path)

print(f"\n{'='*60}")
print(f"Fichiers disponibles : {len(existing_files)}/{len(files_to_upload)}")

if missing_files:
    print(f"\n⚠️  Fichiers manquants ({len(missing_files)}) :")
    for f in missing_files:
        print(f"   • {f}")
    print("\n💡 Ces fichiers seront ignorés lors de l'upload")

📋 PRÉPARATION DES FICHIERS À UPLOADER 


🔍 Vérification des fichiers locaux :

   ✅ ../data/processed/final_recommendations.csv             (42.9 KB)
   ✅ ../data/processed/top20_recommendations.csv             (11.4 KB)
   ✅ ../data/processed/city_weather_scores.csv               (3.8 KB)
   ✅ ../data/processed/top5_destinations.csv                 (0.6 KB)
   ✅ ../data/raw/hotels_top5_all.csv                         (73.6 KB)
   ✅ ../data/processed/rapport_final.html                    (14.5 KB)
   ✅ ../data/processed/carte_tous_hotels.html                (165.6 KB)
   ✅ ../data/processed/carte_top20.html                      (55.3 KB)
   ✅ ../data/processed/dashboard_complet.png                 (813.3 KB)
   ✅ ../data/processed/analysis_fusion.png                   (395.1 KB)

Fichiers disponibles : 10/10


In [9]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 5 : Upload des Fichiers vers S3
# ════════════════════════════════════════════════════════════════════

print("📤 UPLOAD DES FICHIERS VERS S3\n")
print("="*60)

def upload_file(local_path, s3_key):
    """Upload un fichier vers S3 sans ACL."""
    try:
        # Nettoyer la clé S3 (supprimer ../ et .\)
        s3_key_clean = s3_key.replace('../', '').replace('..\\', '').replace('./', '')
        
        # S'assurer qu'il n'y a pas de double slash
        s3_key_clean = '/'.join([p for p in s3_key_clean.split('/') if p and p != '..'])
        
        # Déterminer le Content-Type
        content_types = {
            '.html': 'text/html',
            '.png': 'image/png',
            '.jpg': 'image/jpeg',
            '.jpeg': 'image/jpeg',
            '.csv': 'text/csv',
            '.json': 'application/json',
            '.txt': 'text/plain'
        }
        
        ext = '.' + s3_key_clean.split('.')[-1].lower()
        content_type = content_types.get(ext, 'application/octet-stream')
        
        # Upload SANS ACL (le bucket utilise les bucket policies)
        s3_client.upload_file(
            local_path,
            BUCKET_NAME,
            s3_key_clean,
            ExtraArgs={
                'ContentType': content_type,
                'CacheControl': 'max-age=3600'
                # ✅ ACL supprimé - la politique du bucket gère l'accès public
            }
        )
        
        # URL publique
        url = f"https://{BUCKET_NAME}.s3.{REGION}.amazonaws.com/{s3_key_clean}"
        
        return True, url, s3_key_clean
        
    except Exception as e:
        return False, str(e), s3_key


# Upload
uploaded = 0
failed = 0
urls = {}
uploaded_files = []

print("🚀 Upload en cours...\n")

for local_path, s3_key in existing_files:
    success, result, final_key = upload_file(local_path, s3_key)
    
    if success:
        size_kb = Path(local_path).stat().st_size / 1024
        print(f"✅ {final_key:50s} ({size_kb:.1f} KB)")
        uploaded += 1
        uploaded_files.append(final_key)
        
        # Catégoriser
        if 'rapport' in final_key and final_key.endswith('.html'):
            urls['📄 Rapport Final'] = result
        elif 'carte_tous' in final_key:
            urls['🗺️ Carte Complète'] = result
        elif 'carte_top20' in final_key:
            urls['🏆 Carte Top 20'] = result
        elif 'dashboard' in final_key:
            urls['📊 Dashboard'] = result
        elif 'analysis' in final_key:
            urls['📈 Analyse Fusion'] = result
        elif 'final_recommendations' in final_key:
            urls['📁 Données CSV'] = result
    else:
        print(f"❌ {final_key:50s}")
        print(f"   Erreur : {result}")
        failed += 1

print(f"\n{'='*60}")
print(f"✅ Upload réussi : {uploaded} fichiers")
if failed > 0:
    print(f"❌ Échecs : {failed}")

total_size_mb = sum(Path(f[0]).stat().st_size for f in existing_files) / (1024*1024)
print(f"💾 Taille totale : {total_size_mb:.2f} MB")

📤 UPLOAD DES FICHIERS VERS S3

🚀 Upload en cours...

✅ data/final_recommendations.csv                     (42.9 KB)
✅ data/top20_recommendations.csv                     (11.4 KB)
✅ data/city_weather_scores.csv                       (3.8 KB)
✅ data/top5_destinations.csv                         (0.6 KB)
✅ data/raw/hotels_top5_all.csv                       (73.6 KB)
✅ rapport_final.html                                 (14.5 KB)
✅ cartes/carte_tous_hotels.html                      (165.6 KB)
✅ cartes/carte_top20.html                            (55.3 KB)
✅ images/dashboard_complet.png                       (813.3 KB)
✅ images/analysis_fusion.png                         (395.1 KB)

✅ Upload réussi : 10 fichiers
💾 Taille totale : 1.54 MB


In [10]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 6 : URLs d'Accès Public
# ════════════════════════════════════════════════════════════════════

print("🌐 URLS D'ACCÈS PUBLIC\n")
print("="*60)

if urls:
    print("\n🎯 Fichiers principaux :\n")
    for name, url in urls.items():
        print(f"{name}")
        print(f"   {url}\n")
    
    # URL du bucket
    bucket_url = f"https://{BUCKET_NAME}.s3.{REGION}.amazonaws.com"
    print(f"📦 URL du bucket :")
    print(f"   {bucket_url}\n")
    
    print("="*60)
    print("💡 Testez les URLs dans votre navigateur !")
    print("="*60)
else:
    print("⚠️  Aucune URL générée (aucun fichier principal uploadé)")

🌐 URLS D'ACCÈS PUBLIC


🎯 Fichiers principaux :

📁 Données CSV
   https://251107-140505-jedha-kayak-project.s3.eu-west-3.amazonaws.com/data/final_recommendations.csv

📄 Rapport Final
   https://251107-140505-jedha-kayak-project.s3.eu-west-3.amazonaws.com/rapport_final.html

🗺️ Carte Complète
   https://251107-140505-jedha-kayak-project.s3.eu-west-3.amazonaws.com/cartes/carte_tous_hotels.html

🏆 Carte Top 20
   https://251107-140505-jedha-kayak-project.s3.eu-west-3.amazonaws.com/cartes/carte_top20.html

📊 Dashboard
   https://251107-140505-jedha-kayak-project.s3.eu-west-3.amazonaws.com/images/dashboard_complet.png

📈 Analyse Fusion
   https://251107-140505-jedha-kayak-project.s3.eu-west-3.amazonaws.com/images/analysis_fusion.png

📦 URL du bucket :
   https://251107-140505-jedha-kayak-project.s3.eu-west-3.amazonaws.com

💡 Testez les URLs dans votre navigateur !


In [12]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 7 : Sauvegarde des URLs
# ════════════════════════════════════════════════════════════════════

import pandas as pd

print("💾 SAUVEGARDE DES URLS\n")
print("="*60)

# Sauvegarder dans un fichier texte
urls_file = '../data/processed/aws_s3_urls.txt'

with open(urls_file, 'w', encoding='utf-8') as f:
    f.write("="*60 + "\n")
    f.write("🌐 URLs AWS S3 - Projet Kayak\n")
    f.write("="*60 + "\n\n")
    
    f.write(f"📦 Bucket : {BUCKET_NAME}\n")
    f.write(f"🌍 Région : {REGION}\n\n")
    
    f.write("="*60 + "\n")
    f.write("🎯 FICHIERS PRINCIPAUX\n")
    f.write("="*60 + "\n\n")
    
    for name, url in urls.items():
        f.write(f"{name}\n")
        f.write(f"{url}\n\n")
    
    f.write("="*60 + "\n")
    f.write("📁 TOUS LES FICHIERS\n")
    f.write("="*60 + "\n\n")
    
    base_url = f"https://{BUCKET_NAME}.s3.{REGION}.amazonaws.com"
    
    for _, s3_key in existing_files:
        f.write(f"• {s3_key}\n")
        f.write(f"  {base_url}/{s3_key}\n\n")
    
    f.write("\n" + "="*60 + "\n")
    f.write(f"Généré le : {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*60 + "\n")

print(f"✅ URLs sauvegardées : {urls_file}")

# Sauvegarder aussi en JSON
import json

urls_json = {
    'bucket': BUCKET_NAME,
    'region': REGION,
    'base_url': f"https://{BUCKET_NAME}.s3.{REGION}.amazonaws.com",
    'files': {name: url for name, url in urls.items()},
    'timestamp': pd.Timestamp.now().isoformat()
}

json_file = '../data/processed/aws_s3_urls.json'
with open(json_file, 'w', encoding='utf-8') as f:
    json.dump(urls_json, f, indent=2, ensure_ascii=False)

print(f"✅ URLs sauvegardées (JSON) : {json_file}")

💾 SAUVEGARDE DES URLS

✅ URLs sauvegardées : ../data/processed/aws_s3_urls.txt
✅ URLs sauvegardées (JSON) : ../data/processed/aws_s3_urls.json


In [13]:
# ════════════════════════════════════════════════════════════════════
# CELLULE 8 : Vérification Finale
# ════════════════════════════════════════════════════════════════════

print("✅ VÉRIFICATION FINALE\n")
print("="*60)

# Lister tous les fichiers dans le bucket
try:
    response = s3_client.list_objects_v2(Bucket=BUCKET_NAME)
    
    if 'Contents' in response:
        total_files = response['KeyCount']
        total_size = sum(obj['Size'] for obj in response['Contents'])
        
        print(f"📦 Contenu du bucket '{BUCKET_NAME}' :")
        print(f"   • Total fichiers : {total_files}")
        print(f"   • Taille totale : {total_size / (1024*1024):.2f} MB")
        
        # Grouper par dossier
        folders = {}
        for obj in response['Contents']:
            folder = obj['Key'].split('/')[0] if '/' in obj['Key'] else 'racine'
            folders[folder] = folders.get(folder, 0) + 1
        
        print(f"\n📁 Répartition par dossier :")
        for folder, count in sorted(folders.items()):
            print(f"   • {folder:20s} : {count} fichiers")
    
    else:
        print(f"📭 Le bucket est vide (aucun fichier uploadé)")

except ClientError as e:
    print(f"❌ Erreur lors de la vérification : {e}")

print("\n" + "="*60)
print("🎉 DÉPLOIEMENT S3 TERMINÉ !")
print("="*60)

print("\n🎯 Prochaine étape :")
print("   → Notebook 07_setup_rds.ipynb (Configuration RDS PostgreSQL)")

✅ VÉRIFICATION FINALE

📦 Contenu du bucket '251107-140505-jedha-kayak-project' :
   • Total fichiers : 10
   • Taille totale : 1.54 MB

📁 Répartition par dossier :
   • cartes               : 2 fichiers
   • data                 : 5 fichiers
   • images               : 2 fichiers
   • racine               : 1 fichiers

🎉 DÉPLOIEMENT S3 TERMINÉ !

🎯 Prochaine étape :
   → Notebook 07_setup_rds.ipynb (Configuration RDS PostgreSQL)
